## Preparing Building Dataset

**Authors:** Akash Mahajan, Van-Hai Bui

This code creates a dataset file from raw datasets with time specific, weather, and building-specific features. 

Output files are:
- `data.csv`: CSV file containing final dataset
- `stats.json`: Basic statistics on the `data.csv` that is useful for time series transformation and inverse-transformation

#### Necessary imports

In [ ]:
# Standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

# Custom packages and functions
from utils.data_loader import Dataset
from utils.data_loader import get_dummies, is_weekend, is_work_hour, standardize

##### 1. Loading dataset from internet or local source (Please read in the README.md)

In [ ]:
d = Dataset("metainf.json")
dataset = d.get_data("lawber")
print("Load successful")

df = dataset["data"]
df.head()

##### 2. Merge electricity and hvac consumption

In [ ]:
df_copy = df.copy()
df_copy["elec_kW"] = df_copy["elec_N_kW"] + df_copy["elec_S_kW"]   # Electricity and HVAC are reported separated for North and South wing
df_copy["hvac_kW"] = df_copy["hvac_N_kW"] + df_copy["hvac_S_kW"]

In [ ]:
# Set timestamp index
df_copy["timestamp"] = pd.to_datetime(df_copy["timestamp"])
df_copy.set_index("timestamp", inplace=True)

#### 3. Feature Engineering

##### 3.1. Seasons of the year

In [ ]:
seasons = {1:4, 2:4, 3:1, 
           4:1, 5:1, 6:2, 
           7:2, 8:2, 9:3, 
           10:3, 11:3, 12:4}

season_names = ["is_spring", "is_summer", "is_autumn", "is_winter"]

season_of_year = pd.Series(list(map(lambda x: seasons[x], df_copy.index.month)), 
                           name="season_of_year", index=df_copy.index)

season_of_year = get_dummies(season_of_year, season_names)
df_copy = pd.concat((df_copy, season_of_year), axis=1)

##### 3.2. Weekend status (If current day is weekend)

In [ ]:
day_of_week = df_copy.index.day_of_week + 1
weekend_days = list(map(lambda x: is_weekend(x), day_of_week))
weekend_days = pd.Series(weekend_days, name="is_weekend", index=df_copy.index)

column_names = ["is_weekday", "is_weekend"]
weekend_days = get_dummies(weekend_days, column_names)

df_copy = pd.concat((df_copy, weekend_days), axis=1)

##### 3.3. Building schedule (Building 59 runs between 8 AM to 5 PM)

In [ ]:
hour_of_day = df_copy.index.hour
work_hour = [is_work_hour(d, h) for d, h in zip(day_of_week, hour_of_day)]
work_hour = pd.Series(work_hour, name="is_work_hour", index=df_copy.index)

work_hour = get_dummies(work_hour, ["is_off_duty", "is_on_duty"])
df_copy = pd.concat((df_copy, work_hour), axis=1)

#### 4. Data Preprocessing

##### 4.1. Z-score normalization

$$
\begin{flalign*}
T' = \frac{T - \mu}{\sigma} &&
\end{flalign*}
$$

`mu` and `sigma` are the mean and standard deviation of the time series, respectively.  

In [ ]:
# Select all columns
df_new = df_copy[['is_spring', 'is_summer', 'is_autumn', 'is_winter', 'is_weekday', 'is_weekend', 'is_off_duty', 
                  'is_on_duty', 'Interior_Zone_temp_mean', 'solar_radiation', 'relative_humidity', 
                  'air_temperature', 'wind_speed', 'elec_kW', 'hvac_kW']].copy()

const = 1e-1
df_new["elec_kW"] = np.log(df_new["elec_kW"] + const)  # Target series are log-transformed for reducing seasonality
df_new["hvac_kW"] = np.log(df_new["hvac_kW"] + const)

stats = {}

y = df_new["elec_kW"]   # Z-score normalization of the electricity time series
elec_kW, elec_kW_mean, elec_kW_std = standardize(y)
stats["elec_kW_mean"] = elec_kW_mean
stats["elec_kW_std"] = elec_kW_std

y = df_new["hvac_kW"]   # Z-score normalization of the HVAC time series
hvac_kW, hvac_kW_mean, hvac_kW_std = standardize(y)
stats["hvac_kW_mean"] = hvac_kW_mean
stats["hvac_kW_std"] = hvac_kW_std

df_new["elec_kW"] = elec_kW
df_new["hvac_kW"] = hvac_kW

##### 4.2. Autocorrelation Analysis

- This is important stage to enquire which lag `k` has highest impact on the current time `t`. Therefore, the value which has highest autcorrelation with `t` is used as a lag i.e. `t-k`.
- Instead of checking for each lag `k`, we can use predefined autocorrelation function in Pandas.

In [ ]:
plt.figure(figsize=(10, 4)).set_dpi(128)

y = df_new["hvac_kW"].copy()
pd.plotting.autocorrelation_plot(y, label="hvac_kW")

y = df_new["elec_kW"].copy()
pd.plotting.autocorrelation_plot(y, label="elec_kW")

plt.title("Autocorrelation of Electricity and HVAC with its respective lags upto 168")
plt.xlim([0, 168])
plt.legend()
plt.grid(True)
plt.show()

**From the above analysis, we select both 24-hour (day ago) and 168-hour (week ago) lag for Electricity and HVAC**

In [ ]:
lags = [24, 168]
lag_timediff = {}

for lag in lags: 
    timediff = lag * np.ones(shape = (df_new.shape[0],))   # Stores index for 't-k' value where k E {24, 168}
    # Deal with first 'lag' entries
    for i , diff in enumerate(timediff[:lag + 1]):    # First day (first 24/168 entries) do not have corresponding lags hence use their own value as lag
        timediff[i] = min(diff, i)
    lag_timediff[lag] = timediff

In [ ]:
df_new_index = df_new.index
df_new_copy = df_new.copy()

for target in ["elec_kW", "hvac_kW"]:
    for lag, timediff in lag_timediff.items():
        y = df_new[[target]].copy()
        latest_lag_index = df_new_index - pd.to_timedelta(timediff, unit='H')  # Copy the t-k lag values to corresponding time 't'
        latest_lag = y.loc[latest_lag_index].copy()
        latest_lag.index = df_new_index
        latest_lag = latest_lag.add_prefix(f'lag_{lag}_')
        df_new_copy = pd.concat([latest_lag, df_new_copy], axis=1)

##### Save the data

In [ ]:
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

data_filename = f"{output_folder}/data.csv"
stats_filename = f"{output_folder}/stats.json"

df_new_copy.to_csv(data_filename, index=True)
with open(stats_filename, "w") as file:
    json.dump(stats, file, indent=4)